<a href="https://colab.research.google.com/github/jaggukaka/TTS/blob/main/experiments/huggingfaceexperiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install transformers
! pip install fairseq

     |████████████████████████████████| 3.5 MB 5.2 MB/s 
     |████████████████████████████████| 67 kB 5.1 MB/s 
     |████████████████████████████████| 6.8 MB 36.7 MB/s 
     |████████████████████████████████| 895 kB 60.2 MB/s 
     |████████████████████████████████| 596 kB 67.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.7 MB 5.3 MB/s 
     |████████████████████████████████| 145 kB 64.0 MB/s 
     |████████████████████████████████| 90 kB 8.8 MB/s 
     |████████████████████████████████| 74 kB 2.0 MB/s 
     |████████████████████████████████| 112 kB 74.0 MB/s 


In [2]:
from fairseq.checkpoint_utils import load_model_ensemble_and_task_from_hf_hub
from fairseq.models.text_to_speech.hub_interface import S2THubInterface
from fairseq.models.text_to_speech.hub_interface import TTSHubInterface
import IPython.display as ipd
import torchaudio


models, cfg, task = load_model_ensemble_and_task_from_hf_hub(
    "facebook/xm_transformer_600m-es_en-multi_domain",
    arg_overrides={"config_yaml": "config.yaml"},
)
model = models[0]
generator = task.build_generator(model, cfg)

# speech synthesis
tts_models, tts_cfg, tts_task = load_model_ensemble_and_task_from_hf_hub(
  f"facebook/fastspeech2-en-ljspeech",
  arg_overrides={"vocoder": "griffin_lim", "fp16": False},
)

def performS2S(wav_file):
  # requires 16000Hz mono channel audio
  audio, sample_rate = torchaudio.load(wav_file)

  resampled = torchaudio.transforms.Resample(sample_rate, 16_000)


  sample = S2THubInterface.get_model_input(task, resampled)
  text = S2THubInterface.get_prediction(task, model, generator, sample)


  tts_model = tts_models[0]
  TTSHubInterface.update_cfg_with_data_cfg(tts_cfg, tts_task.data_cfg)
  tts_generator = tts_task.build_generator([tts_model], tts_cfg)

  tts_sample = TTSHubInterface.get_model_input(tts_task, text)
  wav, sr = TTSHubInterface.get_prediction(
      tts_task, tts_model, tts_generator, tts_sample
  )

  ipd.Audio(wav, rate=16_000)

ImportError: ignored